In [1]:
import os
os.chdir("..")
from src.data_cleaning import DataCleaning

In [2]:
cleaning_pipeline = DataCleaning()
df = cleaning_pipeline.run()

2024-03-23 14:45:52 - INFO - Data loaded!
2024-03-23 14:45:52 - INFO - Null values cleaned!
2024-03-23 14:45:52 - INFO - Data types asserted!
2024-03-23 14:45:52 - INFO - Full data cleaned!


In [3]:
df

,CD_ORGAO,NM_ORGAO,ANO_LICITACAO,DS_OBJETO,VL_LICITACAO,DT_HOMOLOGACAO,VL_HOMOLOGADO
0,47500,PM DE ESTÂNCIA VELHA,2017,"CONTRATAÇÃO, EXCLUSIVA PARA EMPRESAS BENEFICIÁ...",72691.16,2018-01-17,72691.16
1,75300,PM DE DERRUBADAS,2017,Aquisição de alimentos.,65808.91,2017-01-23,65808.91
2,88027,PM DE CAPÃO DO CIPÓ,2017,AQUISIÇÃO DE COMBUSTIVEIS PARA MANUTENÇÃO DA F...,1058588.00,2017-02-01,1112500.00
3,88039,PM DE PINHAL DA SERRA,2017,O objeto da presente licitação é o registro de...,1361100.00,2017-01-27,1361100.00
4,69900,PM DE PINHAL,2017,AQUISIÇÃO DE MATERIAIS DE LIMPEZA,28942.06,2017-01-25,28942.06
...,...,...,...,...,...,...,...
567548,40600,PM DE ALVORADA,2021,Registro de preço para aquisição de equipament...,3726826.37,2022-12-26,2407569.55
567549,50100,PM DE IJUÍ,2021,Credenciamento de pessoas jurídicas e/ou físic...,132015.00,NaT,132015.00
567550,55600,PM DE RIO GRANDE,2021,Exames e avaliações de Ecografias,249900.00,NaT,249900.00
567551,63400,PM DE CAPÃO DA CANOA,2021,Registro de Preços para contratação de empresa...,1736650.00,2021-09-24,664900.00


In [4]:
df["NM_ORGAO"].value_counts()

NM_ORGAO
CENTRAL DE LICITAÇÕES DO ESTADO             12216
PM DE PORTO ALEGRE                           7918
PM DE CAPÃO DA CANOA                         6012
CORSAN - CIA. RIOGRANDENSE DE SANEAMENTO     5886
SES - SECRETARIA DA SAÚDE DO RS              5146
                                            ...  
CM DE LINHA NOVA                                2
CM DE NOVO TIRADENTES                           2
CM DE TUNAS                                     2
CM DE CERRO LARGO                               2
CM DE DONA FRANCISCA                            2
Name: count, Length: 1085, dtype: int64

# 3. **Feature Engineering**

Three features will be created to assist in the analysis:

- tp_orgao: Indicates the type of public agency, there are a total of 17 types
- dt_homologacao_mes: Number of the month in which the bidding was approved
- dt_homologacao_semana_do_ano: Number of the week of the year in which the bidding was approved

## 3.1. **tp_orgao**

In [6]:
# Prefeituras Municipais
prefeitura = df[df["NM_ORGAO"].str.contains("PM")]["NM_ORGAO"].to_list()

# Câmaras Municipais
camara = df[df["NM_ORGAO"].str.contains("CM")]["NM_ORGAO"].to_list()

# Polícias
policia = ['POLÍCIA CIVIL', 'BRIGADA MILITAR']

# Bancos
banco = ['BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL',
          'BANCO DO ESTADO DO RIO GRANDE DO SUL S.A.',
          'BANRISUL CARTÕES S.A.',
          'BANRISUL S.A. CORRETORA DE VALORES MOBILIÁRIOS E CÂMBIO',
          'BANRISUL S.A. - ADMINISTRADORA DE CONSÓRCIOS',
          'BADESUL DESENVOLVIMENTO S.A. - AGÊNCIA DE FOMENTO/RS']

# Insituições de Previdência e Assistência Social
prev = ['INST. PREV. ASSIST. SERV. MUN. DE CANOAS',
         'FUND. DE ASSIST. SOCIAL - CAXIAS DO SUL',
         'INST. DE PREV. E ASSIST. DOS SERV. MUNIC. DE N. HAMBURGO',
         'INST. PREV. ASSIST. MUN. - CAXIAS DO SUL',
         'INST. PREV. SERV. PUBL. MUNIC. DE IJUÍ',
         'INST. PREV.  ASSIST. À SAÚDE SERV. PÚBL. MUN. DE S. MARIA',
         'INSTITUTO ERECHINENSE DE PREVIDÊNCIA - IEP',
         'SISTEMA DE PREVIDÊNCIA MUNICIPAL - S. LIVRAMENTO',
         'FUNDAÇÃO DE PREVIDÊNCIA COMPLEMENTAR DO SERV. PÚBLIC. DO ESTADO DO RS',
         'INST. PREV. DOS SERV. PÚBL. MUN. DE SÃO GABRIEL',
         'FUND. DE ASSIST. SOCIAL E CIDADANIA - PORTO ALEGRE',
         'INST. PREV. ASSIST. SERV. MUN. CAMPO BOM',
         'INST. APOS. PENSÕES SERV. MUN. - SÃO LEOPOLDO',
         'INST. PREV. DOS SERV. PÚBL. DO MUNIC. - TRÊS PASSOS',
         'FUND. GAÚCHA DO TRABALHO E AÇÃO SOCIAL',
         'PREV-XANGRI-LÁ',
         'FUND. DE ATENDIMENTO SÓCIO-EDUCATIVO DO RGS',
         'DEP. MUNICIPAL DE HABITAÇÃO - PORTO ALEGRE',
         'INST. PREV. SERV. PÚBL. MUN. DE CACHOEIRINHA - IPREC',
         'FUND. DE PROTEÇÃO ESPECIAL DO RIO GRANDE DO SUL',
         'FUND. VENÂNCIO RAMOS DA SILVA - ITAQUI',
         'INST. MUN. ASSIST. SERV. DE NOVA SANTA RITA']

# Órgaos de Saúde (Hospitais, Secretarias de saúde, etc)
saude = ['FUND. DE SAÚDE PÚBLICA DE NOVO HAMBURGO',
        'FUND. HOSPITAL MUNICIPAL GETÚLIO VARGAS',
        'HOSP. MUNICIPAL SÃO JOSÉ DE IBIRAIARAS',
        'HOSP. NOSSA SRA. DO CARMO - AUTARQUIA MUNICIPAL - TAPES',
        'CONS. INTERM. DE SAÚDE DO VALE DO RIO TAQUARI',
        'CONS. INTERM. SAÚDE DO NOROESTE DO ESTADO DO RS',
        'FUND. HOSPITALAR SANTA TEREZINHA DE ERECHIM',
        'CONS. INTERM. DE SAÚDE - CONISA',
        'FUND. DE SAÚDE PÚBLICA SÃO CAMILO DE ESTEIO - FSPSCE',
        'CONSORCIO DE SAUDE INTER-MUNICIPAL',
        'CONS. INTERM. DE SAÚDE DA REGIÃO DAS MISSÕES',
        'SECRETARIA DA SAÚDE DO RS',
        'INST. MUN. DE ESTRAT.  DE SAÚDE DA FAMÍLIA - IMESF',
        'CAIXA PREST. ASSIST. SERV. SAÚDE DOS SERV. MUN. DE P. FUNDO']

# Empresas de Energia
energia = ['CENTRAIS ELÉTRICAS DE CARAZINHO S/A.',
           'CIA. DE GÁS DO ESTADO DO RIO GRANDE DO SUL',
           'DEP. MUNICIPAL DE ENERGIA - IJUÍ',
           'DEP. MUNICIPAL DE ENERGIA DE IJUÍ - DEMEI GERAÇÃO']

# Empresas de Transporte Urbano
transporte = ['CIA. CARRIS PORTO-ALEGRENSE',
              'EMPRESA PÚBLICA DE TRANSPORTE E CIRCULAÇÃO S/A',
              'DEP. AUTÁRQUICO TRANSP. COLETIVOS - RIO GRANDE',
              'DEP. ESTADUAL DE TRÂNSITO',
              'FUND. PASSO FUNDO DE TURISMO PASSOTUR',
              'EMPRESA GAÚCHA DE RODOVIAS S.A.']

# Empresas de Água/Esgoto
agua_esgoto = ['SERVIÇO AUTÔN. MUN. DE ÁGUA E ESGOTO - CX.SUL',
               'COMUSA - SERVIÇOS DE ÁGUA E ESGOTO DE NOVO HAMBURGO',
               'DEP. DE ÁGUA E ESGOTOS - SANT\'ANA DO LIVRAMENTO',
               'SERVIÇO AUTÔN. DE SANEAMENTO DE PELOTAS',
               'CONS. PÚBL. SAN. BÁSICO DA BACIA HIDROG. DO RIO DOS SINOS',
               'DEP. MUNICIPAL DE ÁGUAS E SANEAMENTO DE IJUÍ - DEMASI',
               'FUND. MUNICIPAL DE SAÚDE DE CANOAS',
               'FUND. MÉD.-HOSP. DR. HONOR T. COSTA - LAVRAS DO SUL',
               'CIA. RIOGRANDENSE DE SANEAMENTO',
               'DEPARTAMENTO DE ÁGUA, ARROIOS E ESGOTO DE BAGÉ',
               'DEP. MUNICIPAL DE ÁGUA E ESGOTOS - PORTO ALEGRE',
               'EMPRESA FARROUPILHENSE DE SANEAMENTO E DESENVOLVIMENTO AMBIENTAL S.A.',
               'ÁGUAS DA NASCENTE - SERV. MUN. DE SANEAMENTO AMBIENTAL',
               'ÁGUA DE IVOTI']

# Universidades
universidade = ['UNIVERSIDADE ESTADUAL DO RIO GRANDE DO SUL']

# Consórcios Intermunicipais
cons_interm = ['CONS. INTERM. DO VALE DO JACUÍ - CI/JACUÍ'
              'CONS. PÚBL. INTERM. P/ ASSUNTOS ESTRATÉGICOS DO G8 - CIPAE G8',
              'CONS. INTERM. - CI - CENTRO SUL',
              'CONS. INTERM. ABRIGO INSTITUCIONAL DA PAZ - CIAIP',
              'CONS. INTERM. DA REGIÃO CENTRO DO ESTADO DO RS - CI/CENTRO STA. MARIA',
              'CONS. INTERM. DA REGIÃO NORDESTE',
              'CONS. INTERM. DE COOP. EM GESTÃO PÚBLICA - CONIGEPU',
              'CONS. INTERM. DE DESENV. SUSTENT. DA SERRA GAÚCHA - CISGA',
              'CONS. INTERM. DE GESTÃO AMPLIADA DA REGIÃO CARBONÍFERA - CIGA',
              'CONS. INTERM. DE SERVIÇOS DO VALE DO RIO PARDO',
              'CONS. INTERM. DO MÉDIO ALTO URUGUAI',
              'CONS. INTERM. DO VALE DO RIO CAÍ - CIS/CAÍ',
              'CONS. INTERM. ESMERALDA / PINHAL DA SERRA',
              'CONS. INTERM. LAR DE ACOLHIMENTO SÃO FRANCISCO',
              'ADM. DE  CONSÓRCIOS  INTERMUNIC. S/A. - CAXIAS DO SUL',
              'CONS. PÚBL. INTERM. DA REGIÃO DO ALTO URUGUAI - CIRAU',
              'CONS. PÚBL. INTERM. DESENV. ECON., SOC. E AMB. MUN. BACIA RIO JAGUARÃO',
              'CONS. DE DESENV. INTERM. DOS MUNIC. DO ALTO JACUÍ',
              'CONS. DE DESENV. SUSTENT. DA QUARTA COLÔNIA - CONDESUS/QUARTA COLÔNIA',
              'CONS. PÚBL. DA ASSOC. DOS MUNIC. DA REGIÃO METR. DE POA - CP GRANPAL',
              'CONS. PÚBL. DO EXTREMO SUL', 'CONS. PÚBL. FRONTEIRA NOROESTE',
              'CONSÓRCIO INTERMUNICIPAL DA REGIÃO DO PLANALTO MÉDIO',
              'CONSÓRCIO PÚBLICO DA ASSOCIAÇÃO DOS MUNICÍPIOS DO VALE DO RIO DOS SINO',
              'CONSÓRCIO PÚBLICO INTERMUNICIPAL DE INCLUSÃO SOCIAL E PRODUTIVA']

# Agências Reguladoras
ag_reguladoras = ['AGÊNCIA EST. DE REGULAÇÃO DOS SERV. PÚBL. DELEG. DO RGS',
                  'AGÊNCIA MUN. REGULAÇÃO  DOS SERV. PÚBL. DELEG. DE S. GABRIEL',
                  'AGÊNCIA REGULADORA DE SERVIÇOS PÚBLICOS DELEGADOS DE SANTA CRUZ DO SUL',
                  'AGÊNCIA REGULADORA DOS SERV. PÚBL. MUNIC. DE ERECHIM']

# Órgaos de Tecnologia
tecnologia = ['CIA. DE PROCESSAMENTO DE DADOS DO RS',
              'EMPRESA MUNICIPAL DE INFORMÁTICA DE PELOTAS',
              'CIA. PROC. DE DADOS DO MUNIC. PORTO ALEGRE']

# Instituições de Cultura
cultura = ['FENAC S.A. FEIRAS E EMPREEND. TURÍSTICOS - N. HAMBURGO',
           'CIA. ÁGUAS TERMAIS MARCELINO RAMOS - TERMASA',
           'FESTA NACIONAL DA UVA TURISMO E EMPREENDIMENTOS S/A.',
           'FUND. CULT. AFIF JORGE SIMÕES FILHO - SÃO SEPÉ',
           'FUND. MUN. DE ARTES DE MONTENEGRO',
           'CIA. RIOGRANDENSE DE ARTES GRÁFICAS']

# Instituições de Agricultura
agricultura = ['ASSOCIAÇÃO SULINA DE CRÉDITO E ASSISTÊNCIA RURAL',
               'CIA. ESTADUAL DE SILOS E ARMAZÉNS',
               'CIA. RIOGRANDENSE DE MINERAÇÃO',
               'INSTITUTO RIOGRANDENSE DO ARROZ',
               'FUND. MUN. DE MEIO AMBIENTE - GRAVATAÍ']

# Órgaos Principais do Estado (Secretarias do Estado, Tribunais, Assembléia, etc)
orgaos_principais = ['PROCURADORIA-GERAL DE JUSTIÇA',
                     'SECRETARIA DA FAZENDA DO RS',
                     'SECRETARIA DA SEGURANÇA PÚBLICA',
                     'SECRETARIA DE JUSTIÇA, CIDADANIA E DIREITOS HUMANOS',
                     'SECRETARIA DO MEIO AMBIENTE E INFRAESTRUTURA',
                     'TRIBUNAL DE CONTAS DO ESTADO DO RS',
                     'TRIBUNAL DE JUSTIÇA MILITAR',
                     'DEFENSORIA PÚBLICA DO ESTADO',
                     'ASSEMBLEIA LEGISLATIVA']

# Outras Instituições
outros = ['SUPERINTENDÊNCIA DO PORTO DE RIO GRANDE',
          'CENTRAL DE LICITAÇÕES DO ESTADO',
          'CENTRAIS DE ABASTECIMENTO DO RIO GRANDE DO SUL S.A.',
          'CIA. DE DESENV. DE CAXIAS DO SUL',
          'CIA. DE DESENV. DE PASSO FUNDO',
          'CIA. INDUSTRIAL E DE DESENV. URBANO - CRUZ ALTA',
          'INST. DE PLANEJAMENTO DE SANTA MARIA',
          'CIA. MUNICIPAL DE URBANISMO - NOVO HAMBURGO']

In [8]:
# Adicionando o tp_orgao (tipo de órgão)
df["TP_ORGAO"] = df["NM_ORGAO"].apply(lambda x: 'prev'              if x in prev              else
                                                        'banco'             if x in banco             else
                                                        'saude'             if x in saude             else
                                                        'camara'            if x in camara            else
                                                        'energia'           if x in energia           else
                                                        'policia'           if x in policia           else
                                                        'cultura'           if x in cultura           else
                                                        'prefeitura'        if x in prefeitura        else
                                                        'transporte'        if x in transporte        else
                                                        'tecnologia'        if x in tecnologia        else
                                                        'agua_esgoto'       if x in agua_esgoto       else
                                                        'cons_interm'       if x in cons_interm       else
                                                        'agricultura'       if x in agricultura       else
                                                        'universidade'      if x in universidade      else
                                                        'ag_reguladoras'    if x in ag_reguladoras    else
                                                        'orgaos_principais' if x in orgaos_principais else 
                                                        'outros' if x in outros else "NAOTEM" )

In [ ]:
len(df["NM_ORGAO"].unique())

In [9]:
df["TP_ORGAO"].value_counts()

TP_ORGAO
prefeitura           484451
NAOTEM                60597
outros                12216
camara                 9125
agua_esgoto             912
cultura                 128
orgaos_principais        74
banco                    46
prev                      2
Name: count, dtype: int64

## testar com polars

In [10]:
# Criar o dicionário de mapeamento
tipo_orgao_map = {
    'prefeitura': prefeitura,
    'camara': camara,
    'policia': policia,
    'banco': banco,
    'prev': prev,
    'saude': saude,
    'energia': energia,
    'agua_esgoto': agua_esgoto,
    'transporte': transporte,
    'agricultura': agricultura,
    'universidade': universidade,
    'cons_interm': cons_interm,
    'ag_reguladoras': ag_reguladoras,
    'tecnologia': tecnologia,
    'cultura': cultura,
    'orgaos_principais': orgaos_principais,
    'outros': outros
}

# Função para atribuir tipo de órgão
def assign_tipo_orgao(orgao):
    for tipo, orgaos_list in tipo_orgao_map.items():
        if orgao in orgaos_list:
            return tipo
    return "NAOTEM"

# Adicionar a coluna tp_orgao
df["TP_ORGAO"] = df["NM_ORGAO"].apply(assign_tipo_orgao)


## 3.2. **dt_homologacao_mes e dt_homologacao_semana_do_ano**

In [25]:
df_lic['dt_homologacao_mes'] = df_lic['dt_homologacao'].dt.month
df_lic['dt_homologacao_semana_do_ano'] = df_lic['dt_homologacao'].dt.isocalendar().week